# 1141_社會資料分析 期末考
學生：劉晏成

本次考試模擬學術研究的真實情境，以量化分析方式，回答「新北市民的政治自由態度，是否會因為
性別、年齡、教育年數、個人收入而有差異？」這個研究問題。

使用變項：
* 性別（a1）
* 年齡（age）
* 年齡分組（agegroup）
* 教育年數（eduy）
* 收入（inc）
* 政治自由態度（P_liberal） （由 g4a – g4i 等 9 題所建構而成）

In [1]:
from load import load_sav
import pandas as pd

df = load_sav("../data/final.sav")

In [2]:
variable_value_labels = df.attrs['variable_value_labels']
column_names = df.attrs['column_names']
column_names_to_labels = df.attrs['column_names_to_labels']

def print_labels(key):
    variable_labels = variable_value_labels.get(key, None)
    column_labels = column_names_to_labels.get(key, None)

    print(f"variable_labels: {variable_labels}")
    print(f"column_labels: {column_labels}")

In [5]:
df_final = df[["a1", "age", 'agegroup', 'eduy', 'inc', 'P_liberal']].copy().rename(columns = { 'a1': 'sex' })

df_final

,sex,age,agegroup,eduy,inc,P_liberal
0,2.0,30.0,1.0,16.0,25000.0,2.857143
1,1.0,31.0,1.0,16.0,35000.0,2.857143
2,2.0,33.0,1.0,16.0,25000.0,2.714286
3,1.0,24.0,1.0,12.0,35000.0,2.285714
4,2.0,59.0,3.0,12.0,5000.0,2.714286
...,...,...,...,...,...,...
863,2.0,58.0,3.0,6.0,0.0,3.285714
864,2.0,38.0,2.0,12.0,0.0,2.857143
865,2.0,68.0,3.0,6.0,0.0,3.285714
866,1.0,39.0,2.0,12.0,35000.0,3.428571


## Q1: 控制教育年數之後，收入和政治自由態度之間的淨相關如何？與未控制之前的零階相關差別為何？

首先選取相關係數的計算方法。由於三個變項皆非順序變項，故採用Peasron 相關係數作為主要統計方法。

In [6]:
df_1 = df_final[["eduy", "inc", 'P_liberal']].copy()

檢查遺漏值狀況

In [7]:
from statistic.missing_value import check_missing_value

check_missing_value(df_1)

,# of Missing Values
eduy,0
inc,4
P_liberal,8


發現資料含遺漏值，由於將影響相關係數結果，剔除遺漏值資料

In [8]:
df_1_cleared = df_1.dropna()
check_missing_value(df_1_cleared)

,# of Missing Values
eduy,0
inc,0
P_liberal,0


In [9]:
from statistic.correlation import zero_order_correlation_matrix, partial_correlation_matrix

檢視零階相關結果

In [10]:
zero_order_correlation_matrix(df_1, ['inc', 'P_liberal'], method='pearson')

,Variable,Statistic,inc,P_liberal
0,inc,Correlation,1.0,0.107
1,inc,Significance (2-tailed),.,0.002
2,inc,df,0,854
3,P_liberal,Correlation,0.107,1.0
4,P_liberal,Significance (2-tailed),0.002,.
5,P_liberal,df,854,0


檢視淨階相關結果

In [12]:
partial_correlation_matrix(df_1, ['inc', 'P_liberal'], 'eduy', method='pearson')

,Control Variable,Variable,Statistic,inc,P_liberal
0,eduy,inc,Correlation,1.0,0.056
1,eduy,inc,Significance (2-tailed),.,0.101
2,eduy,inc,df,0,853
3,eduy,P_liberal,Correlation,0.056,1.0
4,eduy,P_liberal,Significance (2-tailed),0.101,.
5,eduy,P_liberal,df,853,0


### Answer
首先觀察零階相關與淨相關的相關係數：
1. inc 與P_liberal 的零階相關係數為0.107
2. inc 與P_liberal 的淨相關係數為0.056

接下來檢定零階相關與淨相關  
$H_0: 不存在相關$  
$H_1: 存在相關$  
$\alpha = 0.05$

觀察零階相關與淨相關的雙尾顯著值：
1. 零階相關顯著值為0.002
2. 淨相關顯著值為0.101

未加入教育年數的零階相關具有統計顯著相關性，並且相關性為弱相關；而控制教育年數後，收入與政治自由態度並為未具相關性。

可發現儘管在為加入教育年數前，零階相關的統計拒絕虛無假設，通過檢定；也就是顯示收入與政治自由態度具有統計相關性。然而在控制教育年數後，二者便不再具有顯著相關性。這表示教育年數作為干擾變數（confounding variable），共同影響了兩個變相。因此此份報告無法支持收入與政治自由態度具統計相關性。

## Q2: 以政治自由態度作為依變項，教育年數作為自變項，年齡做為調節變項，做同時迴歸；檢驗教育年數對政治自由態度的影響是否受年齡調節。

In [13]:
df_2 = df_final[["eduy", "age", 'P_liberal']].copy()

檢查遺漏值狀況

In [15]:
from statistic.missing_value import check_missing_value

check_missing_value(df_2)

,# of Missing Values
eduy,0
age,0
P_liberal,8


發現資料含遺漏值，由於將影響相關係數結果，剔除遺漏值資料

In [16]:
df_2_cleared = df_2.dropna()
check_missing_value(df_2_cleared)

,# of Missing Values
eduy,0
age,0
P_liberal,0


## Q3: 以政治自由態度作為依變項，性別、年齡分組、教育年數、收入作為自變項，做同時迴歸
1. 討論哪個自變項最有助於我們解釋個人的政治自由態度
2. 檢視此迴歸模型是否有多元共線性的問題

## Q4: 按下列步驟，創造「政治自由態度分層」變項。
1. 將 P_liberal 重新編碼成新變項「政治自由態度分層」（變項名 liberal_3gp）；1- 2.6 分編成新變
項的 1（數值標籤設成「低」）、2.7- 3.0 分編成 2（「中」）、3.1 分以上編成 3（「高」）；
2. 列出「政治自由態度分層」變項的次數分配。

## Q5: 按下列步驟，分析「性別」和「政治自由態度分層」是否有顯著相關
1. 做「性別」和「政治自由態度分層」的交叉表（細格內呈現「百分比」和「調整的標準化」） ；
2. 做卡方分析；
3. 報告分析結果（如果卡方分析結果顯著，則要報告殘差分析結果） 。

## Q6: 按下列步驟，分析「年齡分組」和「政治自由態度分層」是否有顯著相關
1. 做「年齡分組」和「政治自由態度分層」的交叉表（細格內呈現「百分比」和「調整的標準化」） ；
2. 做卡方分析 和 Gamma 分析；
3. 報告分析結果：包含相關顯著性，關連方向（正相關或負相關）、關連強度。